In [1]:
import numpy as np
import  pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from plot_keras_history import plot_history
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
data=pd.read_csv('DATA.csv')

In [3]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [4]:
data['label']=data['label'].replace(2,1)

In [5]:
X=data['combined_result']
y=data['label']
time_series=data['t_dist']

In [6]:
std_X = X.std()
desired_snr = 0.1
std_noise = std_X * desired_snr
noise = np.random.normal(0, std_noise, X.shape)
#noise = np.clip(noise, a_min=0, a_max=None)

In [7]:
data['noisy_feature_column'] = X + noise
noisy_signal=data['noisy_feature_column']

In [116]:
data.head(10)

,t_dist,combined_result,label,noisy_feature_column
0,0.00,0.013,1.0,0.020998
1,0.01,0.000,1.0,0.020699
2,0.02,0.004,1.0,0.007873
3,0.03,0.005,1.0,0.004154
4,0.04,0.000,1.0,0.003660
5,0.05,0.000,1.0,-0.003166
6,0.06,0.000,1.0,-0.005002
7,0.07,0.000,1.0,0.001206
8,0.08,0.000,1.0,0.011198
9,0.09,0.000,1.0,-0.007104


In [9]:
# Extracting sequences from the dataset
sequences=[]
labels=[]
temp=[]

for index,value in y.items():
    if value==1:
        temp.append(noisy_signal[index])
        if (index+1)<len(y) and y[index+1]==0:
            sequences.append(temp)
            labels.append(1)
            temp=[]
    if value==0:
        temp.append(noisy_signal[index])
        if (index+1)<len(y) and y[index+1]==1:
            sequences.append(temp)
            labels.append(0)
            temp=[]

In [10]:
# Checking the Number of events
zero=0
one=0
for value in labels:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

724
725


In [11]:
# Padding and reshaping
X_train=pad_sequences(sequences,padding='post',truncating='post',dtype=float,maxlen=50)

In [12]:
y_train = np.array(labels)
y_train = y_train.reshape(-1, 1)

In [13]:
y_train.shape

(1449, 1)

In [14]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(1449, 50, 1)

In [40]:
# It is because Our each  sequence has max laength of 50 and in output we want to predict a label
Tx=50
Ty=1
repeator = tf.keras.layers.RepeatVector(Tx)
concatenator = tf.keras.layers.Concatenate(axis=-1)
densor1 =tf.keras.layers. Dense(10, activation = "tanh")
densor2 = tf.keras.layers.Dense(1, activation = "relu")
activator = tf.keras.layers.Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor =tf.keras.layers. Dot(axes = 1)

In [41]:
def one_step_attention(a,s_prev):
    # We done this to change s_prev to shape of(m,Tx,n_s) for cocatination with a, because concatenation requires all dimension same accept concat axis in this 
    # case is last one so a and s_prev have same first two dims (m,Tx) but last one is changed
    s_prev=repeator(s_prev)
    # We will here concatenate a and s_prev
    concat=concatenator([a,s_prev])
    # here i will calculate energies with 2 dense layers
    e=densor1(concat)
    energies=densor2(e)
    # we know alpha is softmax of this energy
    alpha=activator(energies)
    # to calculate context vector we take dot product of alpha and a
    context_vector=dotor([alpha,a])
    return context_vector

In [42]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.
post_activation_LSTM_cell = tf.keras.layers.LSTM(n_s, return_state = True) # Please do not modify this global variable.
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [43]:
def modelf(Tx,Ty,n_a, n_s):
    X=tf.keras.layers.Input(shape=(X_train.shape[1],1)) # because embedding layer only demands the sequence length if i give full shape like (m,Tx) the
    # ouput of embedding layer will be 4D which can not be fed into BILSTM
    # hidden state for post LSTM
    s0 = tf.keras.layers.Input(shape=(n_s,), name='s0')
    # cell state for post lstm
    # because we know From CampusX that shape of hidden and cell state of lstm are equal
    c0 = tf.keras.layers.Input(shape=(n_s,), name='c0')
    s=s0
    c=c0
    outputs = []

    a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_a,return_sequences=True),input_shape=(X_train.shape[1],1))(X)

    for t in range(Ty):
        context=one_step_attention(a,s)
        _,s,c=post_activation_LSTM_cell(context,initial_state = [s,c] )
        out = output_layer(s)
        outputs.append(out)



    print(outputs)
    model=tf.keras.models.Model(inputs=[X,s0,c0],outputs=outputs)

    return model

In [44]:
model = modelf(Tx, Ty, n_a, n_s)

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_9')>]


In [45]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001,beta_1=0.9,beta_2=0.999)#0.00001
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = opt, metrics = ['accuracy'])

In [46]:
# For Training Set
m=X_train.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [47]:
model_history=model.fit([X_train, s0, c0], y_train, epochs=300, batch_size=100)#350

Epoch 1/300
15/15 [==============================] - 5s 29ms/step - loss: 0.6914 - accuracy: 0.7322
Epoch 2/300
15/15 [==============================] - 0s 29ms/step - loss: 0.6897 - accuracy: 0.7798
Epoch 3/300
15/15 [==============================] - 0s 30ms/step - loss: 0.6881 - accuracy: 0.7674
Epoch 4/300
15/15 [==============================] - 0s 28ms/step - loss: 0.6864 - accuracy: 0.7764
Epoch 5/300
15/15 [==============================] - 0s 29ms/step - loss: 0.6849 - accuracy: 0.7792
Epoch 6/300
15/15 [==============================] - 0s 27ms/step - loss: 0.6833 - accuracy: 0.7688
Epoch 7/300
15/15 [==============================] - 0s 28ms/step - loss: 0.6818 - accuracy: 0.7640
Epoch 8/300
15/15 [==============================] - 0s 29ms/step - loss: 0.6803 - accuracy: 0.7585
Epoch 9/300
15/15 [==============================] - 0s 27ms/step - loss: 0.6789 - accuracy: 0.7585
Epoch 10/300
15/15 [==============================] - 0s 28ms/step - loss: 0.6774 - accuracy: 0.7550

In [49]:
plot_history(model_history,show_standard_deviation=False,show_average=True)

<IPython.core.display.Javascript object>

(<Figure size 1000x500 with 2 Axes>,
 array([<Axes: title={'center': 'Loss'}, xlabel='Epochs', ylabel='Loss'>,
        <Axes: title={'center': 'Accuracy'}, xlabel='Epochs', ylabel='Accuracy'>],
       dtype=object))

In [176]:
#model.save_weights('Model-Trained-On-Noise-Only.h5')
model.load_weights('Model-Trained-On-Noise-Only.h5')

**Testing On Clean Data**

In [117]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [118]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.shape

(59999, 3)

In [119]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [120]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [121]:
event=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            event.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            event.append(temp)
            labe.append(0)
            temp=[]

In [122]:
# Not a sequence but label is 1 
zero_seq_label_one=[]
for position, sublist in enumerate(event):
    if all(element == 0 for element in sublist):
        if labe[position]==1:
            zero_seq_label_one.append(position)

In [123]:
print(zero_seq_label_one)

[4, 80, 86, 108, 146, 184, 190, 194, 206, 208, 230, 242]


In [124]:
# Replacing those with 0
for value in zero_seq_label_one:
    labe[value]=0

In [125]:
# Is a sequence but label 0
seq_label_zero=[]
for position, sublist in enumerate(event):
    if any(element != 0 for element in sublist):
        if labe[position]==0:
            seq_label_zero.append(position)

In [126]:
print(seq_label_zero)

[1, 23, 43, 69, 77, 85, 143, 151, 161, 167, 175, 217, 245, 251, 269, 285]


In [127]:
# Replacing those with 1
for value in seq_label_zero:
    labe[value]=1

In [128]:
# Checking the Number of events
zero=0
one=0
for value in labe:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

140
149


In [129]:
event=pad_sequences(event,padding='post',truncating='post',dtype=float,maxlen=50)
labe=np.array(labe)
labe=labe.reshape(-1,1)
event=event.reshape(event.shape[0], event.shape[1], 1)

In [130]:
# For 10 Min Data set
m_10=event.shape[0]
s0_10 = np.zeros((m_10, n_s))
c0_10 = np.zeros((m_10, n_s))

In [174]:
predictions_10=model.predict([event, s0_10, c0_10])
predictions_10=np.round(predictions_10)

10/10 [==============================] - 0s 5ms/step


In [175]:
predictions_10_count=np.count_nonzero(predictions_10==1)
actual_10_count=np.count_nonzero(labe==1)
print(predictions_10_count/actual_10_count)

0.6620689655172414


In [133]:
report = classification_report(predictions_10,labe)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      0.73      0.85       191
         1.0       0.66      1.00      0.79        98

    accuracy                           0.82       289
   macro avg       0.83      0.87      0.82       289
weighted avg       0.88      0.82      0.83       289


**Testing On Noisy Data**

In [160]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [161]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.shape

(59999, 3)

In [162]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [163]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [164]:
std_X = x_values.std()
desired_snr = 0.1
std_noise = std_X * desired_snr
noise = np.random.normal(0, std_noise, x_values.shape)

In [165]:
min_10_data['noisy_feature_column'] = x_values + noise
noisy_signal=min_10_data['noisy_feature_column']

In [166]:
event=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(noisy_signal[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            event.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(noisy_signal[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            event.append(temp)
            labe.append(0)
            temp=[]

In [167]:
# Checking the Number of events
zero=0
one=0
for value in labe:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

144
145


In [168]:
event=pad_sequences(event,padding='post',truncating='post',dtype=float,maxlen=50)
labe=np.array(labe)
labe=labe.reshape(-1,1)
event=event.reshape(event.shape[0], event.shape[1], 1)

In [169]:
# For 10 Min Data set
m_10=event.shape[0]
s0_10 = np.zeros((m_10, n_s))
c0_10 = np.zeros((m_10, n_s))

In [170]:
predictions_10 = model.predict([event, s0_10, c0_10])
predictions_10 = np.round(predictions_10)

10/10 [==============================] - 0s 7ms/step


In [171]:
predictions_10_count=np.count_nonzero(predictions_10==1)
actual_10_count=np.count_nonzero(labe==1)
print(predictions_10_count/actual_10_count)

0.6620689655172414


In [172]:
report = classification_report(predictions_10,labe)
print(report)

              precision    recall  f1-score   support

         0.0       0.95      0.71      0.81       193
         1.0       0.61      0.93      0.74        96

    accuracy                           0.78       289
   macro avg       0.78      0.82      0.78       289
weighted avg       0.84      0.78      0.79       289


**Conclusion**
As of the testing results these are some my point of views:
1- As you see while testing on noisy that every time noise is different in real world the case is also same. Every time when you run the noise addition cell
the noise comes different so the accuracy and F1 score also increases or decreases.
2- As results are not quite good predictable because the model is not generalizing well on Train set.
3- The reason is also same here because the noise is different every time and model is not getting generalized well.